In [4]:
import igraph as ig

In [5]:
"""ref_network, beta, mu, rho_0, T_max, T_trans"""


GRAPHS = {
    ## ER
    "ER_500_0.2":ig.Graph.Erdos_Renyi(500, p=0.2),
    #"ER_750_0.4":ig.Graph.Erdos_Renyi(750, p=0.4),
    #"ER_900_0.6":ig.Graph.Erdos_Renyi(1000, p=0.6),
    ## Barabasi
    "BA_500_3":ig.Graph.Barabasi(n=500, m=3),
    #"BA_750_2":ig.Graph.Barabasi(n=750, m=2),
    #"BA_1000_2":ig.Graph.Barabasi(n=1000, m=2),
    ## Watts
    "WS_500_4_0.2":ig.Graph.Watts_Strogatz(dim = 1,size=500, nei=4, p=0.2),
    #"WS_750_4_0.4":ig.Graph.Watts_Strogatz(dim = 1,size=750, nei=4, p=0.4),
    #"WS_900_4_0.6":ig.Graph.Watts_Strogatz(dim = 1,size=900, nei=4, p=0.6),
    ## Forest Fire
    "FF_500_0.99":ig.Graph.Forest_Fire(n=500, fw_prob=0.90),
    #"FF_750_0.99":ig.Graph.Forest_Fire(n=750, fw_prob=0.90),
    #"FF_900_0.99":ig.Graph.Forest_Fire(n=900, fw_prob=0.90),
}

In [8]:
import numpy as np
NET = GRAPHS["ER_500_0.2"]
BETA = 0.02
P = 0.2
SIZE = 500
MU = 0.1
TIME_STEPS = 1000 
PERCENTAGE = 0.9
T_TRANS = int(1000 - TIME_STEPS*PERCENTAGE)
SEED = np.random.seed(42)


## The initial state of the graph
## get the nodes
def initial_state(graph:ig.Graph):
    
    nodes = np.arange(0, len(graph.vs()))
    starting_infected_nodes_idx = list(np.random.choice(nodes, size=int(P*SIZE), replace=False))
    ## Sus
    susceptible_nodes_idx = list(np.setdiff1d(nodes, starting_infected_nodes_idx).tolist())
    ## mask the nodes with the infected and susceptible. Infected nodes are 1, susceptible nodes are 0
    masked_nodes = np.zeros(SIZE)
    masked_nodes[starting_infected_nodes_idx] = 1 ## add the infected nodes
    masked_nodes[susceptible_nodes_idx] = 0 ## add the susceptible nodes
    ## get the neighbors list of all the nodes 
    neighbors = NET.neighborhood(order=1, mode="ALL")
    #arr_neighbors = np.array([np.array(n) for n in neighbors])
    
    payload = {
        
        "nodes":nodes,
        "starting_infected_nodes_idx":starting_infected_nodes_idx,
        "susceptible_nodes_idx":susceptible_nodes_idx,
        "masked_nodes":masked_nodes,
        "neighbors":neighbors,
    }
    return payload, nodes, starting_infected_nodes_idx, susceptible_nodes_idx, masked_nodes, neighbors



In [9]:
## we get the infected ones 
infected_counter = len(infected_nodes_idx)
## for each of the infected nodes, if a random number is larger than the beta, then we add the node to the recovered list 
recovered_nodes = [x for x in infected_nodes_idx if np.random.uniform(0,1) > BETA]
## with the susceptible nodes
## if the neighbor is infected
## we filter the neighbor list, by the infected nodes
infected_neighbors = arr_neighbors[infected_nodes_idx]
## 
infected_neighbors_array = [(masked_nodes[x],infected_nodes_idx[idx]) for idx, x in enumerate(infected_neighbors)]


NameError: name 'infected_nodes_idx' is not defined

In [20]:
import copy
## set the initial state of the graph
data, nodes, starting_infected_nodes_idx, susceptible_nodes_idx, masked_nodes, neighbors = initial_state(NET)

## now for each time step, we update the infected nodes and the susceptible nodes 
new_infected_nodes_idx = copy.deepcopy(starting_infected_nodes_idx)
new_susceptible_nodes_idx = copy.deepcopy(susceptible_nodes_idx)
initial_infected_counter = len(new_infected_nodes_idx)

new_infected_nodes_idx

[296,
 328,
 339,
 249,
 305,
 191,
 298,
 340,
 426,
 289,
 345,
 270,
 85,
 202,
 306,
 87,
 341,
 280,
 103,
 135,
 453,
 273,
 378,
 430,
 2,
 350,
 53,
 162,
 107,
 146,
 33,
 448,
 150,
 263,
 112,
 176,
 422,
 133,
 494,
 265,
 459,
 234,
 74,
 342,
 445,
 285,
 55,
 407,
 69,
 355,
 472,
 255,
 9,
 22,
 399,
 243,
 50,
 327,
 357,
 464,
 473,
 187,
 382,
 388,
 481,
 229,
 410,
 168,
 394,
 412,
 465,
 281,
 19,
 379,
 488,
 397,
 286,
 195,
 99,
 486,
 139,
 238,
 165,
 462,
 439,
 476,
 51,
 385,
 418,
 129,
 197,
 253,
 403,
 0,
 84,
 376,
 151,
 16,
 109,
 100]

In [18]:
infections_per_step = [None]*TIME_STEPS
for idx,time_step in enumerate(range(TIME_STEPS)):
    tobeinfected = []
    toberecovered = []
    infected_counter = len(new_infected_nodes_idx)
    ## iterate over theinfected nodes 
    for infected_node in new_infected_nodes_idx:
        ## if mu is greater than the random number, then we add the node to the to be recovered list 
        chance = np.random.uniform(0,1)
        if chance < MU:
            toberecovered.append(infected_node)
    ## iterate over the neighbors of the positive ones 
    for positive_nodes in new_susceptible_nodes_idx:
        ## if the node is in theinfected index we draw a random number, and if beta is larger, then it will be infected. 
        positive_node_neighbors = neighbors[positive_nodes]
        ## if one of the neighbors is infected, then we draw a random number 
        for neighbor in positive_node_neighbors:
            if neighbor in new_infected_nodes_idx:
                chance = np.random.uniform(0,1)
                if chance < BETA:
                    tobeinfected.append(positive_nodes)
                    break
    ## get thecurrentsituation of the infection 
    infection_rate = len(tobeinfected)/len(nodes)
    print(infection_rate)
    infections_per_step[idx] = infection_rate
    ## first remove the recovered nodes from the infected list
    new_infected_nodes_idx = list(np.setdiff1d(new_infected_nodes_idx, toberecovered).tolist())
    ## then add the new infected nodes
    new_infected_nodes_idx = list(np.union1d(new_infected_nodes_idx, tobeinfected).tolist())
    ## now we update the susceptible nodes
    new_susceptible_nodes_idx = list(np.setdiff1d(new_susceptible_nodes_idx, tobeinfected).tolist())
    
avg_rho = np.mean(infections_per_step[T_TRANS:])

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [17]:
avg_rho

0.0

In [ ]:

        # Update states:
        for infected_node in next_infected_nodes:
            G.nodes[infected_node]["State"] = "I"
        for recovered_node in next_recovered_nodes:
            G.nodes[recovered_node]["State"] = "S"
    # Compute average of infection rate:
    rho_avg = np.mean(step_infection_rate[-(T_max-T_trans):])
    return rho_avg

In [47]:
## make the same as the file SIS.py but vectorized 


# Simulate epidemic propagation given model parameters and network
def epidemic_sim(param):
    ref_network, beta, mu, rho_0, T_max, T_trans = param
    G = ref_network
    # Instantiate infected population rho_0=0.2:
    infection_sample = [random.uniform(0, 1) for i in range(100)]
    infection_list = [infection_sample.index(i) for i in infection_sample if i < rho_0]
    for infected_node in infection_list:
        G.nodes[infected_node]["State"] = "I"
    #
    step_infection_rate = []
    # Start random walk:
    for step in range(T_max):
        # Check states:
        next_infected_nodes = []
        next_recovered_nodes = []
        infection_count = 0
        for node in list(G.nodes):
            if G.nodes[node]["State"] == "I":
                infection_count += 1
                random_mu = random.uniform(0, 1)
                if random_mu < mu:
                    next_recovered_nodes.append(node)
            else:
                for neighbor in G.nodes[node]["Neighbors"]:
                    if G.nodes[neighbor]["State"] == "I":
                        random_mu = random.uniform(0, 1)
                        if random_mu < beta:
                            next_infected_nodes.append(node)
                            break
        # Append infection rate
        infection_rate = infection_count/len(list(G.nodes))
        step_infection_rate.append(infection_rate)
        # Update states:
        for infected_node in next_infected_nodes:
            G.nodes[infected_node]["State"] = "I"
        for recovered_node in next_recovered_nodes:
            G.nodes[recovered_node]["State"] = "S"
    # Compute average of infection rate:
    rho_avg = np.mean(step_infection_rate[-(T_max-T_trans):])
    return rho_avg